# 学习笔记

## 1.BPE(Byte-Pair Encoding)

(参考链接)[https://zhuanlan.zhihu.com/p/20877214508]

---

### 分词的三种主要方式
1. 词级别分词
- 最直观的方式,直接按照完整的词来切分
- 优点:保留了完整的词义
- 缺点:
    - 词表太大,存储和计算成本高
    - 无法处理生僻词或新词(OOV问题)
    - 难以处理词形变化(如:play、playing、played)
2. 字符级别分词
- 将文本切分成单个字符
- 优点:词表小,完全没有OOV问题
- 缺点:
    - 失去了词义信息
    - 文本序列变得很长,增加处理难度
3. 子词级别分词(Subword)
- 介于词和字符之间的折中方案
- 常用词保持完整,罕见词拆分成更小的单位
- 主流算法包括BPE、WordPiece等
- 优点:平衡了词表大小和语义表达能力

### BPE算法原理

BPE分为三步：初始化、预分词(pre-tokenization)和合并

#### 1. 初始化

我们会有一个初始词汇表，包含所有256个可能的字节值，因此初始大小为256

#### 2. 预分词

预分词就是统计文本中相邻字符出现的频率。

- 理论上我们可以直接在原始字节序列上统计相邻自己对的出现频率并合并，但是这样每次就要遍历整个语料库，计算成本高。

- 如果不加成本地跨文本合并字节，可能导致如"dog."和"dog!"这样的词被完全无关的标记。

| 阶段 | 处理结果 | 说明 |
| --- | --- | --- |
| **原始文本** | `Hello, world!` | 包含空格和标点。 |
| **预分词** | `["Hello", ",", "world", "!"]` | **关键一步：** 按照空格和标点切开，确保标点符号不会和单词粘在一起。 |
| **子词切分** | `["Hell", "##o", ",", "world", "!"]` | 在预分词的基础上，进一步拆分为模型词表里的最小单位（Token）。 |

### 3. 合并

以论文中的示例说明：假设语料包含以下文本：  
```
low low low low low  
lower lower widest widest widest  
newest newest newest newest newest newest  
```

并设定<|endoftext|>为特殊标记。  

初始词汇表包括<|endoftext|>和全部256个字节。

- 预分词采用空格分割，得到频次统计：low（5次）、lower（2次）、widest（3次）、newest（6次），可表示为字节元组的频数字典，如`{(b'l', b'o', b'w'): 5, ...}`。

- 随后统计所有相邻字节对的频率：lo（7次）、ow（7次）、we（8次）、er（2次）、wi（3次）、id（3次）、de（3次）、es（9次）、st（9次）、ne（6次）、ew（6次）。

- 其中（es）和（st）频率最高且相同，按字典序选择更大的（st）进行合并。于是，所有包含“st”的词如“widest”和“newest”中的“s”和“t”被合并为新标记“st”。

- 第二轮中，“e”与“st”组合出现9次，成为最高频对，合并为“est”。继续此过程，后续合并依次为“ow”、“low”、“west”、“ne”等。若仅执行6次合并，最终新增标记为['s t', 'e st', 'o w', 'l ow', 'w est', 'n e']。  

- [<|endoftext|>, [...256 BYTE CHARS], st, est, ow, low, west, ne]  

此时，“newest”将被切分为[ne, west]两个标记。这一机制在保持对未知词处理能力的同时，有效压缩了序列长度，提升了模型效率。  

同时我们一直重复合并过程直到词表达到一个预设的值


In [1]:
with open('/Users/aiskure/Desktop/transformer-from-scratch-cs336/data/TinyStories-train.txt', 'r') as f:
    print(f.read(1000)) # 读取前1000个字符

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.
Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."
Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
<|endoftext|>
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.
One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were

### 3 BPE 分词器训练实验  

现在我们在 TinyStories 数据集上训练字节级 BPE 分词器。  
https://huggingface.co/datasets/roneneldan/TinyStories/tree/main  

#### 并行化预分词  

预分词是主要性能瓶颈，可使用 `multiprocessing` 库进行并行加速。建议在确保分块边界位于特殊 token 起始位置的前提下对语料分块。可参考以下链接中的示例代码获取分块边界，用于跨进程任务分配：https://github.com/stanford-cs336/assignment1-basics/blob/main/cs336_basics/pretokenization_example.py。  

该分块方式始终有效，因为我们不会在文档边界执行合并操作。本作业中可始终采用此方法，无需考虑极端情况（如超大语料中不含 标记的情形）。  

#### 预分词前移除特殊 token  

在使用正则表达式（`re.finditer`）进行预分词前，应先从语料（或分块）中移除所有特殊 token。必须确保在特殊 token 处分割文本，防止跨文档合并。例如，对于 `[文档1] [文档2]` 的语料，应在 处分割，分别对两部分进行预分词。可通过 `re.split` 实现，使用 `"|".join(special_tokens)` 作为分隔符（注意使用 `re.escape` 处理特殊字符，避免 `|` 等符号引发问题）。测试用例 `test_train_byte_special_tokens` 将验证该行为。  

#### 优化合并步骤  

朴素的 BPE 训练实现效率较低，因每次合并后需重新遍历所有字节对以找最高频对。实际上，仅与被合并字节对相邻的计数会发生变化。因此，可通过维护字节对计数的索引结构，并在每次合并后增量更新相关计数，显著提升速度。尽管该缓存机制能大幅加速，但 BPE 合并步骤在 Python 中无法并行化。  

#### 低资源/降级提示：性能分析  

建议使用 `cProfile` 或 `scalene` 等工具进行性能分析，定位瓶颈并集中优化关键部分。  

#### 低资源/降级提示：降级训练  

不要直接在完整 TinyStories 数据集上训练。建议先在小规模子集（“调试数据集”）上实验，例如使用包含 2.2 万篇文档的验证集（而非完整的 212 万篇）。这是一种通用开发策略：通过使用更小数据、更小模型来加速迭代。选择调试集规模和超参数时需权衡：应足够大以复现真实瓶颈，又不宜过大导致耗时过长。

---

### 问题（15分）BPE 分词器训练  

**交付要求**：编写一个函数，根据输入的文本文件路径训练（字节级）BPE分词器。你的BPE训练函数应至少处理以下输入参数：  
``` sh
input_path: str BPE分词器训练数据文本文件的路径。  

vocab_size: int 定义最终词汇表最大大小的正整数（包括初始字节词汇表、合并产生的词汇表项和任何特殊token）。  

special_tokens: list[str] 需要添加到词汇表中的字符串列表。这些特殊token不会影响BPE训练过程。  

```

你的BPE训练函数应返回最终的词汇表和合并记录：  

vocab: dict[int, bytes] 分词器词汇表，一个从整数（词汇表中的token ID）到字节（token字节）的映射。  

merges: list[tuple[bytes, bytes]] 训练产生的BPE合并记录列表。每个列表项是一个字节元组(<token1>, <token2>)，表示<token1>与<token2>被合并。  
	参考：https://github.com/weiruihhh/cs336_note_and_hw
